# London Passenger Mode Choice

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

#import packages
import pandas as pd
import numpy as np
import pycmtensor as cmt
from pycmtensor.data import Data
from pycmtensor.expressions import Beta, RandomDraws
from pycmtensor.models import MNL
import aesara
import aesara.tensor as aet

lpmc = pd.read_csv("../data/lpmc.dat", sep="\t")
lpmc = lpmc[lpmc["travel_year"]==2015]
# (0: walk, 1: cycle, 2: public transport, 3:drive)
db = Data(
	df=lpmc,
	choice="travel_mode",
	split=0.8
)

asc_walk = Beta("asc_walk", 0.0, None, None, 1)
asc_cycle = Beta("asc_cycle", 0.0, None, None, 0)
asc_pt = Beta("asc_pt", 0.0, None, None, 0)
asc_drive = Beta("asc_drive", 0.0, None, None, 0)
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
s_time = Beta("s_time", 0.5, None, None, 0)
b_purpose = Beta("b_purpose", 0.0, None, None, 0)
b_licence = Beta("b_licence", 0.0, None, None, 0)
b_car_own = Beta("b_car_own", 0.0, None, None, 0)

rnd_time = RandomDraws('rnd_time', 'lognormal', 100)
U_walk = asc_walk + b_time * db["dur_walking"] + rnd_time(s_time) * db["dur_walking"]
U_cycle = asc_cycle + b_time * db["dur_cycling"] 
U_pt = asc_pt + b_time * (db["dur_pt_rail"] + db["dur_pt_bus"] + db["dur_pt_int"]) + b_cost * db["cost_transit"]
U_drive = asc_drive + b_time * db["dur_driving"] + b_licence * db["driving_license"] + b_cost * (db["cost_driving_fuel"] + db["cost_driving_ccharge"])

U = [U_walk, U_cycle, U_pt, U_drive]
mymodel = MNL(db, locals(), U)

In [ ]:
mymodel.reset_values()
mymodel.train(db, batch_size=32, base_learning_rate=0.01)

In [ ]:
#mixed logit
print(mymodel.results.beta_statistics())
print(mymodel.results.model_statistics())

In [ ]:
mymodel.results.model_robust_correlation_matrix()

In [ ]:
print(mymodel.results.benchmark())

In [ ]:
# predictions
print(mymodel.predict(db, return_choices=False))
print(np.unique(mymodel.predict(db), return_counts=True))